# 1. Librerías

In [24]:
#Utilitario para construir modelos de Machine Learning y Deep Learning
import sagemaker

#Importamos el utilitario para definir el entrenador del algoritmo
from sagemaker.estimator import Estimator

#Librería para manipular los servicios de AWS
import boto3

# 2. Inicio de Sesión sobre SageMaker

In [26]:
#Iniciamos sesión en el servicio de SageMaker
sesion = sagemaker.Session()

#Obtenemos la ejecución en donde estamos trabajando
region = sesion.boto_region_name

#Verificamos
print(region)

#Obtenemos el rol de ejecución de SageMaker
#El rol de ejecución permite a SageMaker usar otros servicios de AWS
rol = sagemaker.get_execution_role()

us-east-1


# 3. Lectura de archivos

In [27]:
#Utilitario para leer archivos de datos
from sagemaker.inputs import TrainingInput

#Bucket en donde se encuentran los archivos
#IMPORTANTE: REEMPLAZAR "XXX" POR TUS INICIALES
bucket = "datasetsbdajac"

#Lectura de datos de entrenamiento
dataTrain = TrainingInput(
    f"s3://{bucket}/data/bodyperformance_dataset_validacion_cruzada/vc3/train", #Ruta del archivo
    content_type = "text/csv", #Formato del archivo
    distribution = "FullyReplicated", #El archivo será copiado en todos los servidores
    s3_data_type = "S3Prefix", #Desde donde se lee el archivo (S3)
    input_mode = "File", #Los registros se encuentran dentro de archivos
    record_wrapping = "None" #Envoltorio de optimización
)

#Lectura de datos de validación
dataTest = TrainingInput(
    f"s3://{bucket}/data/bodyperformance_dataset_validacion_cruzada/vc3/test", #Ruta del archivo
    content_type = "text/csv", #Formato del archivo
    distribution = "FullyReplicated", #El archivo será copiado en todos los servidores
    s3_data_type = "S3Prefix", #Desde donde se lee el archivo (S3)
    input_mode = "File", #Los registros se encuentran dentro de archivos
    record_wrapping = "None" #Envoltorio de optimización
)

# 4. Configuración del Algoritmo

In [28]:
#Definimos el entrenador del algoritmo
entrenador = Estimator(
    image_uri = sagemaker.image_uris.retrieve("xgboost", region, version = "latest"), #Descargamos la implementación del algoritmo desde la región donde trabajamos
    role = rol, #Rol que ejecuta servicios sobre AWS
    instance_count = 1, #Cantidad de servidores de entrenamiento
    instance_type = "ml.m5.large", #Tipo de servidor de entrenamiento
    predictor_type = "classifier", #Tipo de predicción del algoritmo
    sagemaker_session = sesion, #Sesión de SageMaker
    base_job_name = "entrenamiento-prediccion-categorica", #Nombre del job de entrenamiento
    num_classes = 4 #Número de categorías en el label
)

In [29]:
#Configuramos los parametros del algoritmo
entrenador.set_hyperparameters(
    feature_dim = 12, #Cantidad de features
    predictor_type = "classifier", #Indicamos que tipo de predicción es
    normalize_data = "true", #Normalizamos los features
    normalize_label = "false", #NO Normalizamos el label
    num_round = 100, #Número de árboles generados
    max_depth = 7, #Profundidad máxima del arbol
    colsample_bytree = 0.95 #Porcentaje de registros validos para generalizar
)

# 5. Definición de Hiper-Parámetros

In [32]:
#Utilitario para definir rangos de hyper-parámetros que pueden tomar valores continuos
from sagemaker.tuner import CategoricalParameter

In [31]:
#Definimos los hyper-parámetros del modelo
hyperparametros = {
    "eta": CategoricalParameter([0.025, 0.05, 0.1, 0.2, 0.4]) #Ratio de aprendizaje
}

In [33]:
#Utilitario para crear la malla de hyper-parametrización
from sagemaker.tuner import HyperparameterTuner

In [34]:
#Creamos la malla de hyper-parametrización
mallaDeHyperParametros = HyperparameterTuner(
    entrenador, #Entrenador del algoritmo
    "validation:rmse", #Métrica para elegir el mejor modelo
    hyperparametros, #Hyper-parámetros definidos
    objective_type = "Minimize", #Elegimos el modelo con el mayor valor de accuracy
    max_jobs = 5, #Cantidad de procesos de entrenamiento.
    max_parallel_jobs = 5 #Cantidad de procesos de entrenamiento en paralelo
)

# 6. Entrenamiento del Modelo

In [35]:
#Ejecutamos la malla de hyper-parametrización
#TIEMPO: 5 MINUTOS
mallaDeHyperParametros.fit(inputs = {"train": dataTrain, "validation": dataTest})

No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config
No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config


...................................!


# 7. Selección del mejor modelo

In [36]:
#Librería de AWS
import boto3

#Nos conectamos al servicio de SageMaker
sagemakerCliente = boto3.client("sagemaker")

#Obtenemos el mejor modelo entrenado (el del menor error)
nombreDelMejorModelo = mallaDeHyperParametros.best_training_job()

#Verificamos
print(nombreDelMejorModelo)

xgboost-240930-2321-003-38468009


# 8. Estadísticas del Modelo

In [38]:
#Obtenemos la descripción del entrenamiento
descripcionDeEntrenamiento = sagemakerCliente.describe_training_job(TrainingJobName = nombreDelMejorModelo)

#Dentro de la sub-variable "FinalMetricDataList" tenemos las métricas del modelo
descripcionDeEntrenamiento["FinalMetricDataList"]

[{'MetricName': 'train:rmse',
  'Value': 0.39054998755455017,
  'Timestamp': datetime.datetime(2024, 9, 30, 23, 23, 43, tzinfo=tzlocal())},
 {'MetricName': 'validation:rmse',
  'Value': 0.5395089983940125,
  'Timestamp': datetime.datetime(2024, 9, 30, 23, 23, 43, tzinfo=tzlocal())},
 {'MetricName': 'ObjectiveMetric',
  'Value': 0.5395089983940125,
  'Timestamp': datetime.datetime(2024, 9, 30, 23, 23, 43, tzinfo=tzlocal())}]

In [39]:
#Imprimimos sólo la métrica de nuestro interés
for metrica in descripcionDeEntrenamiento["FinalMetricDataList"]:
    if metrica["MetricName"] == "validation:rmse":
        print(metrica["Value"])

0.5395089983940125


In [40]:
#Extraemos los hyper-parametros del modelo
descripcionDeEntrenamiento["HyperParameters"]

{'_tuning_objective_metric': 'validation:rmse',
 'colsample_bytree': '0.95',
 'eta': '0.1',
 'feature_dim': '12',
 'max_depth': '7',
 'normalize_data': 'true',
 'normalize_label': 'false',
 'num_round': '100',
 'predictor_type': 'classifier'}

In [41]:
#Extraemos el learning rate
descripcionDeEntrenamiento["HyperParameters"]["eta"]

'0.1'